# Implementing a Recommender System (Seoul summit 2020)



## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
  1. [Explore](#Explore)
  1. [Clean](#Clean)
  1. [Prepare](#Prepare)
1. [Train Locally](#Train-Locally)
  1. [Define Network](#Define-Network)
  1. [Set Parameters](#Set-Parameters)
  1. [Execute](#Execute)
1. [Train with SageMaker](#Train-with-SageMaker)
  1. [Wrap Code](#Wrap-Code)
  1. [Move Data](#Move-Data)
  1. [Submit](#Submit)
1. [Host](#Host)
  1. [Evaluate](#Evaluate)
1. [Wrap-up](#Wrap-up)

---

## Background

AWS Seoul Summit 2020의 '아직도 파이썬으로 머신러닝하니? 난 SQL로 바로 쓴다' 세션 데모를 위한 SageMaker 추천 모델 학습 과정을 보여주기 위한 노트북입니다. Kaggle의 [eCommerce behavior data from multi category store](https://www.kaggle.com/mkechinov/ecommerce-behavior-data-from-multi-category-store) 데이터를 학습하여 User에게 Product를 제공하는 모델을 만들고 추론 endpoint를 구성합니다. 이 노트북은 SageMaker Example Notebooks으로 제공되는 [Gluon Recommender System](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_applying_machine_learning/gluon_recommender_system/gluon_recommender_system.ipynb) 을 참고하여 작성되었습니다. 모델 훈련/평가/추론에 동일한 방식을 적용하였으며 모델의 정확도 보다는 SageMaker에서 학습한 모델을 Aurora에서 어떻게 이용할 수 있는지를 중점적으로 확인하시기 바랍니다.

---

## Setup

_이 노트북은 ml.p3.2xlarge notebook instance로 만들어서 테스트 되었습니다._

다음의 사항에 주의하셔야 합니다:

- S3 버킷과 profix는 Training에 사용될 데이터 및 model이 저장되는 경로입니다. 해당 버킷은 노트북 인스턴스와 같은 Region에 위치해 있어야합니다.
- 트레이닝에 사용될 데이터에 접근 할 수 있는 적합한 IAM role을 셋팅해서 사용하셔야 정상적으로 동작합니다.

In [ ]:
bucket = '<Your bucket>'
prefix = '<file destination/folder for training data>'

import sagemaker
role = sagemaker.get_execution_role()

실습에 필요한 파이썬 라이브러리들을 로드합니다.

In [ ]:
import os
import mxnet as mx
from mxnet import gluon, nd, ndarray
from mxnet.metric import MSE
import pandas as pd
import numpy as np
import sagemaker
from sagemaker.mxnet import MXNet
import boto3
import json
import matplotlib.pyplot as plt

---

## Data

### Explore

S3에서 실습 데이터를 로컬 스토리지로 가져옵니다. 로컬 트레이닝을 위한 샘플링을 해야하므로 비교적 작은 10월 데이터만 다운로드 합니다.

In [ ]:
!mkdir /tmp/recsys/
!aws s3 mv s3://$bucket/2019-Oct.csv s3://$bucket/$prefix/
!aws s3 mv s3://$bucket/2019-Nov.csv s3://$bucket/$prefix/
!aws s3 cp s3://$bucket/$prefix/2019-Oct.csv /tmp/recsys/

데이터를 [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) 을 이용해서 가공하여 탐색합니다. 이 때 로컬 트레이닝에서 사용할 데이터의 사이즈를 줄이기 위해서 샘플을 추출합니다.

In [ ]:
df = pd.read_csv('/tmp/recsys/2019-Oct.csv', delimiter=',',error_bad_lines=False)
df.head()

예제에 사용되는 데이터셋은 다음과 같은 정보들을 담고 있습니다.:

- `event_time`: 이벤트가 일어난 시각 (UTC)
- `event_type`: 이벤트의 유형 [view, cart, remove_from_cart, purchase]
- `product_id`: Product ID
- `category_id`: Product category ID
- `category_code`: Category의 유의미한 이름 (if present)
- `brand`: Brand name (if present)
- `price`: Product 가격
- `user_id`: Permanent user ID
- `user_session`: User session ID

이 예제에서는 `user_id`, `product_id`, `event_type`만 학습에 사용합니다. `category_id`, `category_code`, `brand`으로 `title` 컬럼을 만들어서 추후 결과 데이터를 확인하기 위해 사용합니다. `event_type`의 값 'view, cart, remove_from_cart, purchase'를 각각 0,1,2,3으로 수치화하여 `event_type_digit`를 생성합니다. 이 때 로컬 트레이닝을 위해서 샘플을 추출하여 데이터 사이즈를 줄입니다.

In [ ]:
df = df[['user_id', 'product_id', 'event_type', 'category_id', 'category_code', 'brand']]
df['event_type_digit'] = df['event_type'].apply(lambda x: 4 if x=='purchase' else 3 if x=='cart' else 2 if x=='remove_from_cart' else 1)

In [ ]:
df.loc[df['category_code'].isnull(), 'title'] = df['brand']
df.loc[df['title'].isnull(), 'title'] = df['category_id']

df = df[['user_id', 'product_id', 'event_type_digit', 'title']]

df = df.sample(n=10000000,replace=True)

쇼핑하는 사람들은 관심 있는 제품들만 보았고 실제로 보는 것보다 적은 수의 제품을 구매하기 때문에 트레이닝에 사용되는 데이터에 대한 정보가 Sparse할 것이라고 예측됩니다. 따라서 long tail에 걸리는 정보들을 제거 할 예정입니다. 이를 위해 확인할 몇 가지 기본 백분위수를 살펴 보겠습니다.

In [ ]:
users = df['user_id'].value_counts()
products = df['product_id'].value_counts()

quantiles = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.96, 0.97, 0.98, 0.99, 1]
print('users\n', users.quantile(quantiles))
print('products\n', products.quantile(quantiles))

### Clean

long tail 데이터를 Cleansing합니다.

In [ ]:
users = users[users >= 5]
products = products[products >= 10]

reduced_df = df.merge(pd.DataFrame({'user_id': users.index})).merge(pd.DataFrame({'product_id': products.index}))

Event 횟수가 5번 이상인 사용자정보와 10번 이상 Event가 발생했던 상품들만 이용해서 고객 및 상품 목록을 다시 작성하겠습니다.

In [ ]:
users = reduced_df['user_id'].value_counts()
products = reduced_df['product_id'].value_counts()

다음으로 각 사용자 및 항목에 번호를 매겨서 고유한 순차적 색인을 제공합니다.

In [ ]:
user_index = pd.DataFrame({'user_id': users.index, 'user': np.arange(users.shape[0])})
product_index = pd.DataFrame({'product_id': products.index, 
                              'item': np.arange(products.shape[0])})

reduced_df = reduced_df.merge(user_index).merge(product_index)
reduced_df.head()

### Prepare

로컬 트레이닝을 위해서 샘플 데이터셋을 테스트와 트레이닝 셋으로 나눕니다.

In [ ]:
test_df = reduced_df.groupby('user_id').last().reset_index()

train_df = reduced_df.merge(test_df[['user_id', 'product_id']], 
                            on=['user_id', 'product_id'], 
                            how='outer', 
                            indicator=True)
train_df = train_df[(train_df['_merge'] == 'left_only')]

Pandas 데이터프레임을 MXNet NDArrays 형태로 변형하고 MXNet Data Iterator에 추가합니다.

In [ ]:
batch_size = 1024

train = gluon.data.ArrayDataset(nd.array(train_df['user'].values, dtype=np.float32),
                                nd.array(train_df['item'].values, dtype=np.float32),
                                nd.array(train_df['event_type_digit'].values, dtype=np.float32))
test  = gluon.data.ArrayDataset(nd.array(test_df['user'].values, dtype=np.float32),
                                nd.array(test_df['item'].values, dtype=np.float32),
                                nd.array(test_df['event_type_digit'].values, dtype=np.float32))

train_iter = gluon.data.DataLoader(train, shuffle=True, num_workers=4, batch_size=batch_size, last_batch='rollover')
test_iter = gluon.data.DataLoader(train, shuffle=True, num_workers=4, batch_size=batch_size, last_batch='rollover')

---

## Train Locally

### Define Network

먼저 행렬 인수 분해 작업의 신경망 버전을 정의해 보겠습니다. 이 경우 네트워크는 매우 간단합니다. 주요 구성 요소는 다음과 같습니다.

- [Embeddings](https://mxnet.incubator.apache.org/api/python/gluon/nn.html#mxnet.gluon.nn.Embedding) 인덱스를 고정 된 크기의 조밀 한 벡터로 바꿉니다. 이 경우에는 64입니다.
- [Dense layers](https://mxnet.incubator.apache.org/api/python/gluon.html#mxnet.gluon.nn.Dense)를 Relu 활성화와 함께 사용합니다.각각의 조밀한 레이어는 임베딩 수와 같은 수의 단위를 갖습니다. RelU 활성화는 행렬 인수 분해에 비선형성을 추가합니다.
- [Dropout layers](https://mxnet.incubator.apache.org/api/python/gluon.html#mxnet.gluon.nn.Dropout)는 오버피팅을 방지하는 데 사용할 수 있습니다.
- user matrix와 procudt matrix의 행렬 곱셈은 평가 행렬의 추정치를 만듭니다.

In [ ]:
class MFBlock(gluon.HybridBlock):
    def __init__(self, max_users, max_items, num_emb, dropout_p=0.5):
        super(MFBlock, self).__init__()
        
        self.max_users = max_users
        self.max_items = max_items
        self.dropout_p = dropout_p
        self.num_emb = num_emb
        
        with self.name_scope():
            self.user_embeddings = gluon.nn.Embedding(max_users, num_emb)
            self.item_embeddings = gluon.nn.Embedding(max_items, num_emb)
            
            self.dropout_user = gluon.nn.Dropout(dropout_p)
            self.dropout_item = gluon.nn.Dropout(dropout_p)

            self.dense_user   = gluon.nn.Dense(num_emb, activation='relu')
            self.dense_item = gluon.nn.Dense(num_emb, activation='relu')
            
    def hybrid_forward(self, F, users, items):
        a = self.user_embeddings(users)
        a = self.dense_user(a)
        
        b = self.item_embeddings(items)
        b = self.dense_item(b)

        predictions = self.dropout_user(a) * self.dropout_item(b)     
        predictions = F.sum(predictions, axis=1)
        return predictions

In [ ]:
num_embeddings = 64

net = MFBlock(max_users=user_index.shape[0], 
              max_items=product_index.shape[0],
              num_emb=num_embeddings,
              dropout_p=0.5)


### Set Parameters

네트워크 가중치를 초기화하고 최적화 매개 변수를 설정합니다.

In [ ]:
# Initialize network parameters
ctx = mx.gpu()
net.collect_params().initialize(mx.init.Xavier(magnitude=60),
                                ctx=ctx,
                                force_reinit=True)
net.hybridize()

# Set optimization parameters
opt = 'sgd'
lr = 0.02
momentum = 0.9
wd = 0.

trainer = gluon.Trainer(net.collect_params(),
                        opt,
                        {'learning_rate': lr,
                         'wd': wd,
                         'momentum': momentum})

### Execute

신경망의 훈련을 수행하는 함수를 정의합니다.

In [ ]:
def execute(train_iter, test_iter, net, epochs, ctx):
    
    loss_function = gluon.loss.L2Loss()
    for e in range(epochs):
        
        print("epoch: {}".format(e))
        
        for i, (user, item, label) in enumerate(train_iter):
                user = user.as_in_context(ctx)
                item = item.as_in_context(ctx)
                label = label.as_in_context(ctx)
                
                with mx.autograd.record():
                    output = net(user, item)               
                    loss = loss_function(output, label)
                    
                loss.backward()
                trainer.step(batch_size)

        print("EPOCH {}: MSE ON TRAINING and TEST: {}. {}".format(e,
                                                                   eval_net(train_iter, net, ctx, loss_function),
                                                                   eval_net(test_iter, net, ctx, loss_function)))
    print("end of training")
    return net

네트워크를 평가하는 함수를 정의합니다.

In [ ]:
def eval_net(data, net, ctx, loss_function):
    acc = MSE()
    for i, (user, item, label) in enumerate(data):
        
            user = user.as_in_context(ctx)
            item = item.as_in_context(ctx)
            label = label.as_in_context(ctx)
            predictions = net(user, item).reshape((batch_size, 1))
            acc.update(preds=[predictions], labels=[label])
   
    return acc.get()[1]

트레이닝을 진행해봅니다. 약 10분정도 소요됩니다. (p3.2xlarge 사용 시)

In [ ]:
%%time

epochs = 3

trained_net = execute(train_iter, test_iter, net, epochs, ctx)

#### Early Validation

훈련 오류가 감소하는 것을 확인할 수 있고 이 때 검증 정확도는 올라갑니다. 모델이 개별 사용자들을 어떻게 예측하는지 확인합니다. user6번에 대해서 진행합니다.

In [ ]:
product_index['u6_predictions'] = trained_net(nd.array([6] * product_index.shape[0]).as_in_context(ctx), 
                                              nd.array(product_index['item'].values).as_in_context(ctx)).asnumpy()
product_index.sort_values('u6_predictions', ascending=False)

다른 사용자들에 적용한 결과 값과 비교합니다. user7을 사용합니다.

In [ ]:
product_index['u7_predictions'] = trained_net(nd.array([7] * product_index.shape[0]).as_in_context(ctx), 
                                              nd.array(product_index['item'].values).as_in_context(ctx)).asnumpy()
product_index.sort_values('u7_predictions', ascending=False)

예측 값은 두 사용자 간에 다르지만 user7도 user6과 비교하여 유사한 상위 (및 하위) 항목이 나타납니다. 이를 확인하기 위해 전체 집합에 대한 상관 관계를 살펴 보겠습니다.

In [ ]:
product_index[['u6_predictions', 'u7_predictions']].plot.scatter('u6_predictions', 'u7_predictions')
plt.show()

---

## Train with SageMaker

로컬 트레이닝 결과를 보면 user6과 user7에 대한 예측 값의 상관 관계를 확인할 수 있습니다. 
이제 SageMaker를 활용하여 노트북 인스턴스 외부에서 트레이닝을 하면 더 큰 데이터를 대상으로 보다 효율적으로 작업을 실행할 수 있습니다. 

### Wrap Code

SageMaker의 사전 구축 된 MXNet 컨테이너를 사용하려면 위의 코드를 파이썬 스크립트로 래핑해야합니다. 이에 대한 자세한 문서는 여기서 찾을 수 있습니다. [here](https://github.com/aws/sagemaker-python-sdk#mxnet-sagemaker-estimators):

사전에 작성된 recommmender python code를 다운로드 받은 뒤 내용을 확인합니다.

In [ ]:
!wget "https://raw.githubusercontent.com/elbanic/summit_2020_demo/master/sagemaker-notebook/recommender.py"
!cat recommender.py

### Submit

이제, 우리는 세이지 메이커 파이썬 SDK에서 MXNet estimator 만들 수 있습니다. 이를 위해서 다음을 작성해야합니다.
1. SageMaker 트레이닝 클러스터의 인스턴스 유형 및 개수입니다. SageMaker의 MxNet 컨테이너는 분산 GPU Training을 지원하므로 원하는 경우 여러개의 ml.p2 또는 ml.p3 인스턴스로 쉽게 설정할 수 있습니다.
1. 트레이닝 데이터셋이 저장되어 있는 S3 경로와 모델을 저장하기 위한 S3 경로에 접근할 수 있는 Role을 설정해야 합니다.
1. 신경망에 설정하는 Hyperparameter 값들을 입력해야 합니다.

`.fit()`을 사용하면 SageMaker Training job이 생성되고 학습에 사용하는 instance가 구동됩니다. AWS 계정에 SageMaker 학습을 위한 instance의 limit 값을 설정하셔야합니다. Training이 끝나면 model은 S3에 저장됩니다.

In [ ]:
opt = 'sgd'
lr = 0.02
momentum = 0.9
wd = 0.

m = MXNet('recommender.py', 
          py_version='py3',
          role=role, 
          train_instance_count=1, 
          train_instance_type="ml.p3.2xlarge",
          output_path='s3://{}/{}/output'.format(bucket, prefix),
          hyperparameters={'num_embeddings': 64, 
                           'opt': opt, 
                           'lr': lr, 
                           'momentum': momentum, 
                           'wd': wd,
                           'epochs': 10},
         framework_version='1.1')

m.fit({'train': 's3://{}/{}/'.format(bucket, prefix)})

---

## Host

트레이닝이 끝난 모델을 사용하기 위해서 endpoint에 이를 배포합니다.

In [ ]:
predictor = m.deploy(initial_instance_count=1, 
                     instance_type='ml.m4.xlarge')
predictor.serializer = None

endpoint가 설정되고 나면 test를 해봅니다. user6의 Event type 값을 확인합니다.

*이 작업은 HTTP POST 요청으로도 수행 가능합니다. 여기서는 test를 위해서 SageMaker Python SDK에 있는 `.predict()` 함수를 사용합니다.*

In [ ]:
predictor.predict('512505687,44600062')

---

## Wrap-up

이번 예제를 통해서 간단하게 각 고객들의 특정 제품에 대한 선호도를 딥러닝 모델을 이용해서 예측해보았습니다. 다음은 이 모델을 Aurora에서 함수 형태로 호출하여 사용하는 방법에 대해서 알아보겠습니다.